# Introduction

This notebook is a simple python implementation of Word and Paradigm (WP) morphology for Totonac verbs. It is intended to be used with the data file `totonac-unimorph.txt` (a version of the Totonac data we have collected in UniMorph format), but could—in principle—be adapted to any UniMorph dataset.

## Instructions

Your goal is to construct a morphological and phonological grammar that maximizes the accuracy of your predictions regarding Totonac verbs forms. Your predictor will be presented with each of the lemmas and sets of properties in sequence and will generate a wordform for each based on the inputs and the rules (which form **grammars**).

In [425]:
import re
from unicodedata import normalize

## Codes for natural classes
These regexes represent natural classes (vowels, consonants, and sequences of vowels and consonsants) that can be reused in morphological and phonological rules.

In [426]:
V = "(a|e|i|o|u)\u0301?'?"
C = "(ch|lh|p|t|k|q|m|n|s|j|x|h|l|y|w)"
X = "(.+)"

## Morphological grammar
The morphological grammar goes here. The one provided **is not correct**, but gives you an example you can follow and a foundation on which you can build. Each block is a list of rules. Each rule is a tuple consisting of a set of properties, a pattern (as a regex) and a replacement (using the Python regex syntax). The order of the rules in a block doesn't matter. Rules will always be applied from most specific (most properties) to least specific (fewest properties).

In [427]:
blocks = [
    [
        ({'3', 'P'}, f'{X}', r"\1qó:'y"),
    ],
    [
        ({'1', 'S'}, f'{X}', r'k\1'),
        ({'1', 'EXCL'},  f'{X}', r'k\1'),
    ],
]

## The phonological grammar
The phonological grammar goes here. It consists of a sequence of `(pattern, repl)` tuples. The rules are applied in order.

In [428]:
phonology = [
    ("kk", "jk"),
    ("^([^\u0301aeiou]+[aeiou])([^\u0301]+)$", "\\1\u0301\\2"),
    ("^([^aeiou]*[aeiou])\u0301([^aeiou]*)$", "\\1\\2"),
]

## Interpreters
The following functions change morphological and phonological grammars into functions that can be applied to forms and properties.

In [429]:
def block_to_function(block):
    block = sorted(block, key=lambda x: len(x[0]), reverse=True)
    def f(prop, form):
        for rule_prop, pattern, repl in block:
            if rule_prop.issubset(prop):
                form = re.sub(pattern, repl, form)
                break
        return normalize('NFD', form)
    return f

In [430]:
def blocks_to_function(blocks):
    def f(prop, form):
        for block in blocks:
            bfunc = block_to_function(block)
            form = bfunc(prop, form)
        return normalize('NFD', form)
    return f

In [431]:
def phonology_to_function(phonology):
    def f(form):
        for (pattern, repl) in phonology:
            form = re.sub(pattern, repl, form)
        return form
    return f

## Putting It All Together
This is the code that implements all of the bookkeeping. It prints the input to `stdout`, but with an extra column with the predicted form (if it differs from the reference).

In [432]:
morph_grammar = blocks_to_function(blocks)
phon_grammar = phonology_to_function(phonology)

with open('totonac-unimorph.txt') as f:
    total, correct = 0, 0
    for line in f:
        total += 1
        line = normalize('NFD', line.strip())
        if not line:
            print()
            continue
        lemma, form, prop = line.split('\t')
        propset = set(prop.split(';'))
        pred =  phon_grammar(morph_grammar(propset, lemma))
        pred = normalize('NFD', pred)
        if pred == form:
            correct += 1
            print('\t'.join([lemma, form, prop]))
        else:
            print('\t'.join([lemma, form, prop, pred]))
    print(f'Accuracy: {correct/total}')

mas	kmas	V;1;S;IPFV
mas	mása'	V;2;S;IPFV
mas	mas	V;3;S;IPFV
mas	masá:'w	V;1;P;INCL;IPFV
mas	kmasá:'w	V;1;P;EXCL;IPFV
mas	masá:'ti't	V;2;P;IPFV
mas	masqó:'y	V;3;P;IPFV
mas	kmaslh	V;1;S;PFV
mas	mast	V;2;S;PFV
mas	maslh	V;3;S;PFV
mas	masw	V;1;P;INCL;PFV
mas	kmasw	V;1;P;EXCL;PFV
mas	másti't	V;2;P;PFV
mas	masqó:'lh	V;3;P;PFV
mas	kmasní:'t	V;1;S;PRF
mas	masní:'ta'	V;2;S;PRF
mas	masní:'t	V;3;S;PRF
mas	masni:'tá'w	V;1;P;INCL;PRF	masní:'tá'w
mas	kmasni:'tá'w	V;1;P;EXCL;PRF	kmasní:'tá'w
mas	masni:'tá'nti't	V;2;P;PRF		á'ná'nti't
mas	masqo:'ní:'t	V;3;P;PRF	masqó:'yní:'t
mas	kmásmah	V;1;S;PROG
mas	máspa:'t	V;2;S;PROG	máspa:
mas	másmah	V;3;S;PROG
mas	masmá:'w	V;1;P;INCL;PROG	masmahá:'w
mas	kmasmá:'w	V;1;P;EXCL;PROG	kmasmahá:'w
mas	maspá:'ti't	V;2;P;PROG	máspa:ti't
mas	masma:qó:'lh	V;3;P;PROG	masqó:'ymah

lhtuj	klhtuj	V;1;S;IPFV
lhtuj	lhtúja'	V;2;S;IPFV
lhtuj	lhtuj	V;3;S;IPFV
lhtuj	lhtujá:'w	V;1;P;INCL;IPFV
lhtuj	klhtujá:'w	V;1;P;EXCL;IPFV
lhtuj	lhtujá:'ti't	V;2;